### Introduction

Ce notebook explique le projet complet : de l'exploration des données à la création et au déploiement d'un modèle prédictif avec FastAPI et l'utilisation de SHAP pour explicabilité.
L'objectif est de comprendre chaque étape et de garder une trace pour référence future.

### Étape 1 : Chargement et Exploration des Données

Nous commençons par charger les données et analyser leur contenu pour comprendre les caractéristiques disponibles.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Charger les données
print("Chargement des données...")
df = pd.read_csv('cartest.csv')
print(df.head())

# Description des données
print("Description des données :")
print(df.describe())

# Visualisation des données
sns.pairplot(df)
plt.show()

Les données contiennent des informations sur des voitures d'occasion : 
- `year` : année de fabrication
- `km_driven` : kilométrage
- `fuel` : type de carburant
- `transmission` : type de transmission (manuelle ou automatique)
- `owner` : type de propriétaire précédent
- `seller_type` : type de vendeur (individuel, concessionnaire, etc.)
- `selling_price` : prix de vente

### Étape 2 : Préparation des Données

Nous préparons les données en transformant les colonnes catégoriques et en manipulant les colonnes nécessaires.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encodage des colonnes catégoriques
encoder = LabelEncoder()
df['fuel'] = encoder.fit_transform(df['fuel'])
df['transmission'] = encoder.fit_transform(df['transmission'])
df['owner'] = encoder.fit_transform(df['owner'])
df['seller_type'] = encoder.fit_transform(df['seller_type'])

# Vérification des données après encodage
print("Données après transformation :")
print(df.head())

Les colonnes catégoriques ont été encodées pour que les modèles les comprennent. Par exemple :
- `Diesel` devient 0, `Petrol` devient 1.

### Étape 3 : Création et Entraînement du Modèle

Nous utilisons un modèle de régression simple pour prédire le prix de vente des voitures.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Séparation des données
X = df.drop('selling_price', axis=1)
y = df['selling_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création du modèle
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Évaluation du modèle
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print(f"Performance sur l'ensemble d'entraînement : {train_score}")
print(f"Performance sur l'ensemble de test : {test_score}")

Un modèle `RandomForestRegressor` a été entraîné pour prédire le prix de vente.
Les performances montrent la capacité du modèle à s'adapter aux données.

### Étape 4 : Enregistrement du Modèle avec MLflow

MLflow est utilisé pour gérer les modèles. Cela permet d'enregistrer, de versionner et de suivre les performances des modèles de manière centralisée.

In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("Car Price Prediction")

with mlflow.start_run():
    mlflow.sklearn.log_model(model, artifact_path="random_forest_model")
    mlflow.log_metric("train_score", train_score)
    mlflow.log_metric("test_score", test_score)

print("Modèle enregistré avec succès dans MLflow.")

### Étape 5 : Déploiement avec FastAPI

FastAPI est utilisé pour déployer le modèle sous forme d'API. Cela permet à d'autres applications de faire des prédictions en envoyant des requêtes HTTP.

In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/status")
def status():
    return {"status": "Le modèle est prêt pour les prédictions."}

@app.post("/predict")
def predict(year: int, km_driven: int, fuel: int, transmission: int, owner: int, seller_type: int):
    input_data = pd.DataFrame([[year, km_driven, fuel, transmission, owner, seller_type]],
                              columns=['year', 'km_driven', 'fuel', 'transmission', 'owner', 'seller_type'])
    prediction = model.predict(input_data)
    return {"predicted_price": prediction[0]}

### Étape 6 : Ajout de SHAP pour l'Explicabilité

SHAP est une bibliothèque qui permet de comprendre les décisions des modèles. Le graphique `summary_plot` montre quelles caractéristiques influencent le plus les prédictions.

In [ ]:
import shap

explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

shap.summary_plot(shap_values, X_test)

### Conclusion

Ce projet illustre tout le cycle de vie d'un modèle machine learning : exploration des données, création, entraînement, enregistrement, déploiement, et explicabilité.
Les outils utilisés comme MLflow et SHAP rendent ce processus robuste et transparent.
Prochaines étapes : essayer d'autres modèles et améliorer l'interface utilisateur avec une documentation API plus détaillée.